In [ ]:
!find . -delete
# !git clone https://github.com/sunnynexus/search-o1
!git clone https://github.com/mindscope-world/Search-o1
!ls -la

Cloning into 'Search-o1'...
remote: Enumerating objects: 385, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 385 (delta 68), reused 76 (delta 32), pack-reused 263 (from 1)
Receiving objects: 100% (385/385), 4.52 MiB | 33.81 MiB/s, done.
Resolving deltas: 100% (185/185), done.
total 12
drwxr-xr-x 3 root root 4096 Mar 16 18:34 .
drwxr-xr-x 5 root root 4096 Mar 16 15:03 ..
drwxr-xr-x 6 root root 4096 Mar 16 18:34 Search-o1


In [ ]:
!sed -i '/^pyext==0.7/s/^/# /' requirements.txt # the pyext command is causing dependency install errors

sed: can't read requirements.txt: No such file or directory


In [ ]:
!pip3 install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [ ]:
!wget -q https://github.com/idavidrein/gpqa/raw/refs/heads/main/dataset.zip
!unzip -P deserted-untie-orchid -d GPQA dataset.zip > /dev/null 2>&1
!ls -la GPQA/dataset
!rm dataset.zip.* # removes any extra files from multiple runs

total 8528
drwxr-xr-x 2 root root    4096 Nov 21  2023 .
drwxr-xr-x 3 root root    4096 Mar 16 18:34 ..
-rw-r--r-- 1 root root    6148 Nov 21  2023 .DS_Store
-rw-r--r-- 1 root root 1373492 Nov 21  2023 gpqa_diamond.csv
-rw-r--r-- 1 root root   26636 Nov 21  2023 gpqa_experts.csv
-rw-r--r-- 1 root root 4091077 Nov 21  2023 gpqa_extended.csv
-rw-r--r-- 1 root root 3214603 Nov 21  2023 gpqa_main.csv
-rw-r--r-- 1 root root     241 Nov 21  2023 license.txt
rm: cannot remove 'dataset.zip.*': No such file or directory


In [ ]:
"""Data preprocess for GPQA
Data Link: https://huggingface.co/datasets/Idavidrein/gpqa
"""
import csv
import json
import random
from tqdm import tqdm

# Paths to data
split = 'diamond'  # diamond, main, extended
data_path = f'./GPQA/dataset/gpqa_{split}.csv'
output_path = f'./GPQA/{split}.json'

# Define the keys we want to keep
keys_to_keep = [
    'id',
    'Question',
    'Subdomain',
    'High-level domain',
    'Correct Answer',
    'Incorrect Answer 1',
    'Incorrect Answer 2',
    'Incorrect Answer 3'
]

filtered_data = []
with open(data_path, mode='r', encoding='utf-8') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for idx, row in enumerate(tqdm(csv_reader), 0):
        # Add id field
        row['id'] = idx
        # Create new dictionary with only desired keys
        filtered_row = {key: row[key] for key in keys_to_keep}

        # Extract answers and shuffle them
        answers = [
            ('Correct Answer', filtered_row['Correct Answer']),
            ('Incorrect Answer 1', filtered_row['Incorrect Answer 1']),
            ('Incorrect Answer 2', filtered_row['Incorrect Answer 2']),
            ('Incorrect Answer 3', filtered_row['Incorrect Answer 3'])
        ]
        random.shuffle(answers)

        # Assign new choices A, B, C, D in order and determine the correct choice
        choices = ['A', 'B', 'C', 'D']
        formatted_answers = []
        correct_choice = None
        for i, (label, answer) in enumerate(answers):
            choice = choices[i]
            formatted_answers.append((choice, answer))
            if label == 'Correct Answer':
                correct_choice = choice

        # Update the Question field
        formatted_choices = "\n".join([f"({choice}) {answer}" for choice, answer in formatted_answers])
        filtered_row['Question'] = f"{filtered_row['Question']} Choices:\n{formatted_choices}\n"

        # Add the Correct Choice field
        filtered_row['Correct Choice'] = correct_choice

        # Append the updated row to filtered_data
        filtered_data.append(filtered_row)

# Write the updated data to JSON
with open(output_path, mode='w', encoding='utf-8') as json_file:
    json.dump(filtered_data, json_file, indent=4, ensure_ascii=False)

198it [00:00, 8808.39it/s]


In [ ]:
"""Data preprocess for MATH500
Data Link: https://huggingface.co/datasets/HuggingFaceH4/MATH-500
"""
import pandas as pd

df = pd.read_json("hf://datasets/HuggingFaceH4/MATH-500/test.jsonl", lines=True)
df.head()

,problem,solution,answer,subject,level,unique_id
0,"Convert the point $(0,3)$ in rectangular coord...",We have that $r = \sqrt{0^2 + 3^2} = 3.$ Also...,"\left( 3, \frac{\pi}{2} \right)",Precalculus,2,test/precalculus/807.json
1,Define\n\[p = \sum_{k = 1}^\infty \frac{1}{k^2...,We count the number of times $\frac{1}{n^3}$ a...,p - q,Intermediate Algebra,5,test/intermediate_algebra/1994.json
2,"If $f(x) = \frac{3x-2}{x-2}$, what is the valu...",$f(-2)+f(-1)+f(0)=\frac{3(-2)-2}{-2-2}+\frac{3...,\frac{14}{3},Algebra,3,test/algebra/2584.json
3,How many positive whole-number divisors does 1...,First prime factorize $196=2^2\cdot7^2$. The ...,9,Number Theory,3,test/number_theory/572.json
4,The results of a cross-country team's training...,Evelyn covered more distance in less time than...,\text{Evelyn},Algebra,2,test/algebra/1349.json


In [ ]:
"""Convert Parquet to JSON (AIME)
Data Link: https://huggingface.co/datasets/AI-MO/aimo-validation-aime
"""
df = pd.read_parquet("hf://datasets/AI-MO/aimo-validation-aime/data/train-00000-of-00001.parquet")

# Filter the DataFrame to keep only rows where '2024_AIME' appears in the 'url' column
filtered_df = df[df['url'].str.contains('2024_AIME', na=False)]

# Print the first few rows of the filtered DataFrame to confirm
print(filtered_df.head())

# Export to a JSON file with indentation
json_file = "/aime_2024.json"
filtered_df.to_json(json_file, orient='records', force_ascii=False, indent=4)

print(f"Filtered data has been saved to {json_file[:5]}")

    id                                            problem  \
60  60  Every morning Aya goes for a $9$-kilometer-lon...   
61  61  Let $ABC$ be a triangle inscribed in circle $\...   
62  62  Each vertex of a regular octagon is independen...   
63  63  Define $f(x)=|| x|-\tfrac{1}{2}|$ and $g(x)=||...   
64  64  Let $p$ be the least prime number for which th...   

                                             solution answer  \
60  $\frac{9}{s} + t = 4$ in hours and $\frac{9}{s...    204   
61  From the tangency condition we have $\let\angl...    113   
62  Notice that the question's condition mandates ...    371   
63  If we graph $4g(f(x))$, we see it forms a sawt...    385   
64  If \(p=2\), then \(4\mid n^4+1\) for some inte...    110   

                                                  url  
60  https://artofproblemsolving.com/wiki/index.php...  
61  https://artofproblemsolving.com/wiki/index.php...  
62  https://artofproblemsolving.com/wiki/index.php...  
63  https://artofproblem

In [ ]:
import os
import pandas as pd
import json

# Create directory if missing
os.makedirs('./data/AIME', exist_ok=True)

df = pd.read_parquet("hf://datasets/AI-MO/aimo-validation-aime/data/train-00000-of-00001.parquet")
filtered_df = df[df['url'].str.contains('2024_AIME', na=False)]

# Save to correct path
filtered_df.to_json('./data/AIME/test.json', orient='records', indent=4)

In [ ]:
# """Convert Parquet to JSON (AIME)
# Data Link: https://huggingface.co/datasets/AI-MO/aimo-validation-amc
# """
# import pandas as pd

# df = pd.read_parquet("hf://datasets/AI-MO/aimo-validation-aime/data/train-00000-of-00001.parquet")

# # 打印 DataFrame 的前几行查看数据
# print(df.head())

# # 导出为 JSON 文件，并加上缩进
# json_file = "/amc_2022_2023.json"
# df.to_json(json_file, orient='records', force_ascii=False, indent=4)

   id                                            problem  \
0   0  Quadratic polynomials $P(x)$ and $Q(x)$ have l...   
1   1  Three spheres with radii $11$, $13$, and $19$ ...   
2   2  Let $ABCD$ be a parallelogram with $\angle BAD...   
3   3  For any finite set $X$, let $| X |$ denote the...   
4   4  Let $S$ be the set of all rational numbers tha...   

                                            solution answer  \
0  Let $R(x)=P(x)+Q(x).$ Since the $x^2$-terms of...    116   
1  This solution refers to the Diagram section.\n...    756   
2  Let's redraw the diagram, but extend some help...    150   
3  Let's try out for small values of $n$ to get a...    245   
4  $0.\overline{abcd}=\frac{abcd}{9999} = \frac{x...    392   

                                                 url  
0  https://artofproblemsolving.com/wiki/index.php...  
1  https://artofproblemsolving.com/wiki/index.php...  
2  https://artofproblemsolving.com/wiki/index.php...  
3  https://artofproblemsolving.com/wiki/

In [ ]:
import os
import pandas as pd

# Create directory structure
os.makedirs('/kaggle/working/Search-o1/data/AIME/', exist_ok=True)

# Load and filter data
df = pd.read_parquet("hf://datasets/AI-MO/aimo-validation-aime/data/train-00000-of-00001.parquet")
filtered_df = df[df['url'].str.contains('2024_AIME', na=False)]

# Rename columns to match expected keys
filtered_df = filtered_df.rename(columns={
    'problem': 'Question',  # Ensure this matches your source column name
    'answer': 'answer'
})

# Add ID column
filtered_df['id'] = filtered_df.index

# Select and save ONLY required fields
filtered_df[['id', 'Question', 'answer']].to_json(
    './data/AIME/test.json',
    orient='records',
    indent=4,
    force_ascii=False  # Preserve special characters
)

In [ ]:
"""Data preprocess for AMC
"""
import csv
import json
from tqdm import tqdm

test_path = '/amc_2022_2023.json'
output_path = '/kaggle/working/Search-o1/data/AIME/test.json'

data_list = []
with open(test_path, 'r') as file:
    data = json.load(file)
    id = 0
    for line in tqdm(data):
        if '2023' not in line['url']:
            continue
        data_list.append({
        'id': current_id,
        'Question': line['problem'],  # WAS MAPPED TO 'problem' INSTEAD OF 'Question'
        'answer': str(int(line['answer'])),
        'url': line['url']
    })
        id += 1

# Write the updated data to JSON
with open(output_path, mode='w', encoding='utf-8') as json_file:
    json.dump(data_list, json_file, indent=4, ensure_ascii=False)

100%|██████████| 90/90 [00:00<00:00, 510808.34it/s]


In [ ]:
"""Data preprocess for LiveCodeBench
Data Link: https://huggingface.co/datasets/livecodebench/code_generation_lite
"""

"""Data preprocess for LiveCodeBench using Hugging Face dataset"""
import json
from tqdm import tqdm
from datetime import datetime
from datasets import load_dataset

def is_valid_date(date_str):
    """
    Check if the given date string is within the range from August 1, 2024, to November 30, 2024.

    Args:
        date_str (str): The date string in the format "%Y-%m-%dT%H:%M:%S".

    Returns:
        bool: True if the date is within the specified range, False otherwise.
    """
    try:
        date = datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%S")
    except ValueError:
        return False

    start_date = datetime(2024, 8, 1)
    end_date = datetime(2024, 11, 30)
    return start_date <= date <= end_date

# Load the dataset from Hugging Face
dataset = load_dataset('livecodebench/code_generation_lite', split='test')

output_path = '/content/test-LiveCodeBench.json'

data_list = []
seen_questions = set()
current_id = 0

# Process each example in the dataset
for example in tqdm(dataset, desc='Processing dataset'):
    contest_date = example.get('contest_date')
    if not contest_date or not is_valid_date(contest_date):
        continue

    question_content = example.get('question_content')
    if not question_content:
        continue

    if question_content in seen_questions:
        continue

    seen_questions.add(question_content)

    data_list.append({
        'id': current_id,
        'Question': question_content,
        'question_title': example.get('question_title', ''),
        'contest_date': contest_date,
        'difficulty': example.get('difficulty', ''),
        'public_test_cases': example.get('public_test_cases', [])
    })

    current_id += 1

# Write the processed data to JSON file
try:
    with open(output_path, mode='w', encoding='utf-8') as json_file:
        json.dump(data_list, json_file, indent=4, ensure_ascii=False)
    print(f"Data successfully written to {output_path}")
except Exception as e:
    print(f"Failed to write data to {output_path}: {e}")

Processing dataset: 100%|██████████| 880/880 [00:23<00:00, 38.14it/s] 

Data successfully written to /content/test-LiveCodeBench.json


In [ ]:
!pip install triton pdfplumber

In [ ]:
!pip install pyext

In [ ]:
!pip show pyext

Name: pyext
Version: 0.7
Summary: Simple Python extensions.
Home-page: 
Author: Ryan Gonzalez
Author-email: kirbyfan64sos@gmail.com
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: 
Required-by: 


In [ ]:
# Install newer compatible versions
!pip install --force-reinstall \
    torch==2.2.1 \
    torchvision==0.17.1 \
    --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121
  Using cached https://download.pytorch.org/whl/cu121/torch-2.2.1%2Bcu121-cp310-cp310-linux_x86_64.whl (757.3 MB)
  Using cached https://download.pytorch.org/whl/cu121/torchvision-0.17.1%2Bcu121-cp310-cp310-linux_x86_64.whl (7.0 MB)
  Using cached https://download.pytorch.org/whl/filelock-3.13.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached https://download.pytorch.org/whl/typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached https://download.pytorch.org/whl/sympy-1.13.1-py3-none-any.whl (6.2 MB)
  Using cached https://download.pytorch.org/whl/networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
  Using cached https://download.pytorch.org/whl/Jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached https://download.pytorch.org/whl/fsspec-2024.6.1-py3-none-any.whl.metadata (11 kB)
  Using cached https://download.pytorch.org/whl/cu121/nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 M

In [ ]:
!pip install vllm==0.4.1  # Works with PyTorch 2.2.1

  Using cached numpy-2.0.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
Using cached numpy-2.0.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (19.5 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.2
    Uninstalling numpy-2.1.2:
      Successfully uninstalled numpy-2.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
catboost 1.2.7 requires numpy<2.0,>=1.16.0, but you have numpy 2.0.2 which is incompatible.
cupy-cuda12x 12.2.0 requires numpy<1.27,>=1.20, but you have numpy 2.0.2 which is incompatible.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.0.2 which is incompatible.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, but you have scipy 1.15.2 which is incompatible.
langchain 0.3.12 requires async-timeout<5.0.0,>=4.0.0; python_version < "3.11", but you have async-timeout 5.

In [ ]:
!pip install transformers==4.46.0

In [ ]:
# Clean environment setup
!pip install --force-reinstall \
    "numpy==1.26.4" \
    "scikit-learn==1.4.0" \
    "nltk==3.8.1" \
    --no-build-isolation

  Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached scikit_learn-1.4.0-1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached nltk-3.8.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached scipy-1.15.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached regex-2024.11.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
Using cached scikit_learn-1.4.0-1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.1 MB)
Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
Using cac

In [ ]:
# --model_path "Qwen/Qwen1.5-32B-Chat"  # For Option 1
# --model_path "Qwen/Qwen2-32B-Instruct"  # For Option 2

# For Chinese support:
# --model_path "Qwen/Qwen1.5-1.8B-Chat"

# For English-optimized:
# --model_path "mistralai/Mistral-7B-v0.1"  # Actually 7B but runs well on T4

# Extremely lightweight:
# --model_path "google/gemma-2b-it"

In [ ]:
!python /kaggle/working/Search-o1/scripts/run_search_o1.py \
    --dataset_name aime \
    --split test \
    --max_search_limit 5 \
    --max_turn 10 \
    --top_k 10 \
    --max_doc_len 3000 \
    --use_jina True \
    --model_path "Qwen/Qwen1.5-1.8B-Chat" \
    --jina_api_key "jina_4c1e46dadf9042f0b9d6758a64629797SNS4CgjXH3k9w6D2xzsOuXHy8nSI" \
    --search_engine duckduckgo

-----------------------
Using aime test set.
-----------------------
WARNING 03-16 18:39:52 config.py:1011] Casting torch.bfloat16 to torch.float16.
2025-03-16 18:39:54,320	INFO worker.py:1821 -- Started a local Ray instance.
INFO 03-16 18:39:56 llm_engine.py:98] Initializing an LLM engine (v0.4.1) with config: model='Qwen/Qwen1.5-1.8B-Chat', speculative_config=None, tokenizer='Qwen/Qwen1.5-1.8B-Chat', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=2, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0)
INFO 03-16 18:40:00 utils.py:608] Found nccl from library /root/.config/vllm/nccl/cu12/libnccl.so.2.18.1
(RayWorkerWrapper pid=3392) INFO 03-16 18:40:00 utils.p